In [21]:
import pandas as pd
import numpy as np
import sklearn
import panel as pn
import re
from bokeh.plotting import figure, show
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter
pn.extension(comms='vscode')

In [22]:
#import the data dictionary using a custom module
import crash_dictionary
crash_info = crash_dictionary.CrashDict('./data/crash_dictionary.csv')
display(crash_info.get_labels(['gcs','source']))
display(crash_info.get_details('cause'))
crash_info.get_df()

,Labels
Name,
gcs,Glasgow Coma Score Total
source,Method of Transmission of Entry Form to CC


,Labels,Units,Levels,Class,Storage,NAs
Name,,,,,,
cause,Main Cause of Death,NaN,7.0,NaN,integer,17118


,Labels,Units,Levels,Class,Storage,NAs
Name,,,,,,
entryid,Unique Numbers for Entry Forms,NaN,NaN,integer,integer,0
source,Method of Transmission of Entry Form to CC,NaN,5.0,NaN,integer,0
trandomised,Date of Randomization,NaN,NaN,Date,double,0
outcomeid,Unique Number From Outcome Database,NaN,NaN,integer,integer,80
sex,NaN,NaN,2.0,NaN,integer,1
age,NaN,NaN,NaN,NaN,integer,4
injurytime,Hours Since Injury,NaN,NaN,numeric,double,11
injurytype,NaN,NaN,3.0,NaN,integer,0
sbp,Systolic Blood Pressure,mmHg,NaN,integer,integer,320


## Cleaning happens elsewhere!
Check to see if the loaded dataframe has any missing values. If so, go back to the cleanup notebook, deal with the missing values, and export a clean csv to load.

In [34]:
def check_na(my_df):
    for non_na_count in my_df.count():
        if non_na_count != len(my_df):
            raise ValueError("No NA values are allowed! Clean the data first.")
    print("No NA values found. This df is ready for analysis!")


In [35]:
## test check_na()
try:
    test_df = pd.read_stata('./data/crash2.dta')
    check_na(test_df)
    print("If this line is visible, the test fails!")
except Exception as e:
    print("If the errow below tells you to go clean the data, this test passes!")
    print(f">>> Error: {e}")
finally:
    del test_df

If the errow below tells you to go clean the data, this test passes!
>>> Error: No NA values are allowed! Clean the data first.


## Load the data

In [39]:
## Load data
cleaned_csv_path = "./cleaned_data/cleaned_crash_data.csv"
crash_df = pd.read_csv(cleaned_csv_path)
try: 
    check_na(crash_df)
    display(crash_df.head(3))
    display(crash_df.describe())
except Exception as e:
    print(e)

No NA values found. This df is ready for analysis!


,Unnamed: 0,entryid,source,trandomised,outcomeid,sex,age,injurytime,injurytype,sbp,...,bstroke,bbleed,bmi,bgi,bloading,bmaint,btransf,bvii,boxid,packnum
0,1,2,electronic CRF by email,16585.0,190.0,female,27.0,1.0,blunt,100.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2001,28
1,2,3,electronic CRF by email,16581.0,4.0,male,30.0,1.0,blunt,70.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,2011,21
2,4,5,electronic CRF by email,16585.0,1154.0,female,23.0,4.0,penetrating,80.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2011,23


,Unnamed: 0,entryid,trandomised,outcomeid,age,injurytime,sbp,rr,cc,hr,...,bstroke,bbleed,bmi,bgi,bloading,bmaint,btransf,bvii,boxid,packnum
count,14013.000000,14013.000000,14013.000000,14013.000000,14013.000000,14013.000000,14013.000000,14013.000000,14013.000000,14013.000000,...,14013.000000,14013.000000,14013.000000,14013.000000,14013.000000,14013.000000,14013.000000,14013.000000,14013.000000,14013.000000
mean,10282.924499,10312.232855,17640.933348,10236.617427,34.196104,2.914110,100.782559,22.807964,3.106758,103.821666,...,0.004282,0.074003,0.002070,0.009919,0.997574,0.986013,0.480340,0.001784,5169.218511,51.606437
std,5843.879062,5864.734981,413.293738,5818.172525,13.970844,2.281599,24.220707,6.184919,1.554011,20.040477,...,0.065297,0.261785,0.045446,0.099104,0.049200,0.117441,0.499631,0.042202,2503.796064,25.078755
min,1.000000,2.000000,16575.000000,4.000000,14.000000,0.100000,10.000000,1.000000,1.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2001.000000,21.000000
25%,5159.000000,5166.000000,17343.000000,5125.000000,23.000000,1.000000,86.000000,19.000000,2.000000,90.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,2760.000000,27.000000
50%,10515.000000,10543.000000,17715.000000,10342.000000,30.000000,2.000000,100.000000,22.000000,3.000000,104.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,4507.000000,45.000000
75%,15354.000000,15403.000000,17975.000000,15266.000000,42.000000,4.000000,112.000000,26.000000,4.000000,120.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,8241.000000,83.000000
max,20206.000000,20270.000000,18285.000000,20199.000000,99.000000,72.000000,250.000000,96.000000,30.000000,220.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9065.000000,98.000000


## TODO: PCA_ANALYSIS